In [1]:
import pandas as pd
import numpy as np
import glob

Cargamos todos los archivos Excel y creamos un dataframe único con todos ellos.

In [2]:
big_df = pd.DataFrame() 
    
for filename in glob.glob("./Rating_*.xlsx"):

    print ("READING DATABASE: "+filename)
    
    df = pd.read_excel(open(filename,'rb'), sheet_name="Resultados",header = None) #Reading SABI Export without index  
    df.columns = ['id', 'nombre_x', 'nif', 'nombre', 'provincia', 'calle', 'telefono', 'web', 'desc_actividad', 'cnae', 'cod_consolidacion', 'rating_grade_h2', 'rating_grade_h1', 'rating_grade_h0', 'rating_numerico_h2', 'rating_numerico_h1', 'rating_numerico_h0', 'modelo_propension_h2', 'modelo_propension_h1', 'modelo_propension_h0', 'guo_nombre', 'guo_id_bvd', 'guo_pais', 'guo_tipo', 'estado_detallado', 'fecha_cambio_estado', 'fecha_constitucion', 'p10000_h0', 'p10000_h1', 'p10000_h2', 'p20000_h0', 'p20000_h1', 'p20000_h2', 'p31200_h0', 'p31200_h1', 'p31200_h2', 'p32300_h0', 'p32300_h1', 'p32300_h2', 'p40100_mas_40500_h0', 'p40100_mas_40500_h1', 'p40100_mas_40500_h2', 'p40800_h0', 'p40800_h1', 'p40800_h2', 'p49100_h0', 'p49100_h1', 'p49100_h2']
    df['h0_anio'] = 2017     
    df = df.fillna('')
    df=df.drop(df.index[0]) #Dropping SABI variable names.
    df['nif'] = df.nif.str.upper() #CONVERTING cif INTO UPPERCASE

    for partida in ['p10000_h0', 'p10000_h1', 'p10000_h2', 'p20000_h0', 'p20000_h1', 'p20000_h2', 'p31200_h0', 'p31200_h1', 'p31200_h2', 'p32300_h0', 'p32300_h1', 'p32300_h2', 'p40100_mas_40500_h0', 'p40100_mas_40500_h1', 'p40100_mas_40500_h2', 'p40800_h0', 'p40800_h1', 'p40800_h2', 'p49100_h0', 'p49100_h1', 'p49100_h2']:
        df[partida] = pd.to_numeric(df[partida], errors='coerce').fillna(0)- 0.005

        df['nif_normalizado'] = df['nif'].str[-8:]    
    big_df = big_df.append(df, ignore_index=True)     


df = big_df
df['target_status'] = [0 if i in ['Activa', ''] else 1 for i in df['estado_detallado']] # 0 si Activa, 1 si algo raro!


READING DATABASE: ./Rating_2.xlsx
READING DATABASE: ./Rating_1.xlsx
READING DATABASE: ./Rating_3.xlsx
READING DATABASE: ./Rating_5.xlsx
READING DATABASE: ./Rating_4.xlsx


In [3]:
df.head(5)

,id,nombre_x,nif,nombre,provincia,calle,telefono,web,desc_actividad,cnae,...,p40100_mas_40500_h2,p40800_h0,p40800_h1,p40800_h2,p49100_h0,p49100_h1,p49100_h2,h0_anio,nif_normalizado,target_status
0,1,Novagrup Property Development SL. (EN Liquidac...,B82219999,Novagrup Property Development SL. (EN Liquidac...,Madrid,"CAMINO ZARZUELA, 15",+34 91/7957593,www.grupopinar.es,PROMOCION Y CONSTRUCCION DE INMUEBLES DE TODO ...,4110,...,4787.41500,0.00000,-86.48800,-79.02800,0.00000,-327182.94800,-1906.86100,2017,82219999,1
1,2,Mecaplast Iberica Sociedad Anonima. (Extinguida),A45374501,Mecaplast Iberica Sociedad Anonima. (Extinguida),Toledo,BARRIO ESTACION (CR L-15 SN),,www.mecaplast.com,"EL DESARROLLO INDUSTRIALIZACION, FABRICACION, ...",2229,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,2017,45374501,1
2,3,Union Salinera De España SA,A08009375,Union Salinera De España SA,Madrid,"CALLE CARRERA SAN JERONIMO, 15 - PISO 2",+34 91/4363100,www.unionsalinera.es,Extracción y comercialización al por mayor de ...,0893,...,41286.53300,-644.35400,-705.34900,-748.56400,5912.64900,5535.46300,7728.16900,2017,08009375,0
3,4,Transportes Antonio Diaz Hernandez Sociedad Li...,B35062926,Transportes Antonio Diaz Hernandez Sociedad Li...,Las Palmas de Gran Canaria,"CALLE LAS MIMOSAS ((POL. INDUSTRIAL ARINAGA)),...",+34 92/8181201,www.canarybus.com,LA REALIZACION DE LA ACTIVIDAD DE ARRENDAMIENT...,4931,...,27914.44575,-3045.63306,-2781.84965,-2548.51962,3056.55084,3587.81983,2523.16170,2017,35062926,0
4,5,Axpe Consulting SL,B84184548,Axpe Consulting SL,Madrid,"CALLE ARTURO SORIA, 122 - A - B",+34 91/5901960,www.axpe.com,Gestión y explotación de consultoría informática.,6202,...,39400.11780,-98.02583,-111.25868,-133.74270,321.62079,667.30630,1833.56269,2017,84184548,0


Eliminamos las columnas que no nos interesan/no incluiremos en el modelo. Exploramos todas para ver si incluímos alguna además del CNAE

In [4]:
#Comprobamos que las columnas nombre_x y nombre son idénticas
df['nombre_x'].tolist() == df['nombre'].tolist()

True

In [5]:
print(df.columns.tolist())

['id', 'nombre_x', 'nif', 'nombre', 'provincia', 'calle', 'telefono', 'web', 'desc_actividad', 'cnae', 'cod_consolidacion', 'rating_grade_h2', 'rating_grade_h1', 'rating_grade_h0', 'rating_numerico_h2', 'rating_numerico_h1', 'rating_numerico_h0', 'modelo_propension_h2', 'modelo_propension_h1', 'modelo_propension_h0', 'guo_nombre', 'guo_id_bvd', 'guo_pais', 'guo_tipo', 'estado_detallado', 'fecha_cambio_estado', 'fecha_constitucion', 'p10000_h0', 'p10000_h1', 'p10000_h2', 'p20000_h0', 'p20000_h1', 'p20000_h2', 'p31200_h0', 'p31200_h1', 'p31200_h2', 'p32300_h0', 'p32300_h1', 'p32300_h2', 'p40100_mas_40500_h0', 'p40100_mas_40500_h1', 'p40100_mas_40500_h2', 'p40800_h0', 'p40800_h1', 'p40800_h2', 'p49100_h0', 'p49100_h1', 'p49100_h2', 'h0_anio', 'nif_normalizado', 'target_status']


In [6]:
#Observamos los valores de las columnas intermedias
df.iloc[:,10:27].head()

,cod_consolidacion,rating_grade_h2,rating_grade_h1,rating_grade_h0,rating_numerico_h2,rating_numerico_h1,rating_numerico_h0,modelo_propension_h2,modelo_propension_h1,modelo_propension_h0,guo_nombre,guo_id_bvd,guo_pais,guo_tipo,estado_detallado,fecha_cambio_estado,fecha_constitucion
0,U2,C+,C,CCC-,1.60146,0.955406,2.84609,58.8915,62.834,20.0593,,,,,Disolución: La empresa se encuentra en situaci...,2017-07-04 00:00:00,1998-12-22 00:00:00
1,U1,CC+,CC+,CC+,2.74606,2.74606,2.74606,16.5209,16.5209,16.5209,,,,,Extinción: La empresa se encuentra en situació...,2008-11-13 00:00:00,1996-04-17 00:00:00
2,U1,AAA,AA+,AA+,9.74341,9.3533,9.15894,43.3137,46.7253,47.8693,FRANCOIS HUBERT,WW*110165663908,n.d.,Una o más personas físicas o familias,Activa,1923-11-19 00:00:00,1923-11-19 00:00:00
3,U1,BBB-,BBB,BBB,6.3817,6.83405,6.57509,61.826,63.6515,60.1374,TRANSPORTES ANTONIO DIAZ HERNANDEZ SOCIEDAD LI...,ESB35062926,ES,Empresa,Activa,1982-04-20 00:00:00,1982-04-20 00:00:00
4,U1,A-,BBB,BBB-,7.24484,6.56021,6.27973,49.4438,50.7863,48.5947,MANUEL JESUS REVUELTA LUCERA,WW*110021276101,n.d.,Una o más personas físicas o familias,Activa,2004-10-29 00:00:00,2004-10-26 00:00:00


In [7]:
df['cod_consolidacion'].value_counts()

U1    40469
C2     5972
U2     3464
C1       95
Name: cod_consolidacion, dtype: int64

In [8]:
df['guo_pais'].value_counts()

ES      24182
        16624
n.d.     2226
US       1175
DE        937
        ...  
CU          1
LT          1
DO          1
MU          1
BS          1
Name: guo_pais, Length: 88, dtype: int64

In [9]:
df['guo_tipo'].value_counts()

Empresa                                               21425
                                                      16624
Una o más personas físicas o familias                  8363
Empresa financiera                                     1063
Fondos mutuos & de pensiones/Nominee/Trust/Trustee      808
Autoridades públicas, Estado, Gobierno                  517
Banco                                                   493
Seguro                                                  239
Fundaciones/Institutos de investigación                 228
Firmas Private Equity                                   198
Empleados/Administradores/Directores                     28
Venture capital                                          14
Name: guo_tipo, dtype: int64

Del manual de SABI

- C2: Cuentas consolidadas de una empresa que tiene también cuentas no consolidadasincluidas en SABI.
- C1: Cuentas consolidadas de una empresa que  no tiene cuentas no consolidadas enSABI.
- U2: Cuentas no consolidadas de una empresa que tiene también cuentas consolidadasincluidas en SABI.
- U1: Cuentas no consolidadas de una empresa que no tiene cuentas consolidadas enSABI.

Es una variable muy desbalanceada, pero se podría mantener y hace ohe o le. Son igualmente interesantes la calificación de rating de los años anteriores o el tipo y la fecha de constitución para calcular la antigüedad de la empresa.

Sin embargo, al tratarse de una plataforma fintech, no queremos aumentar el número de campos que solicitamos al usuario. Idealmente, en caso de querer incluir dichos datos en el modelo deberíamos incluir todos los datos corporativos en la base de datos para que el usuario incluyese únicamente, por ejemplo, el CIF.

Igualmente, sería interesante enriquecer el modelo no solo con los resultados del año anterior al cierre, como se realizará, sino también con datos de años anteriores. Del mismo modo, para evitar pedir el doble de datos al usuario y al tratarse de un proyecto didáctico se ha decidido omitirlo, pero podría implementarse de la misma forma descrita en el párrafo anterior.

Mantendremos por tanto las columnas empleadas en el modelo pre-existente en la plataforma junto con el código CNAE y la variable objetivo.

Mostramos a continuación el significado de las mismas:

#### Ebita Margin - Ebitda / Turn over (Ventas)
- p49100: Profit (Resultado del ejercicio)
- p40800: Amortization (Amortización) 
- p40100: Sales Turnover (Ingresos de Explotación)
- p40500: Other sales (Otros Ingresos)
 

#### Total Debt / Ebita 
- p31200: Short Term Debt / Deuda a corto plazo
- p32300: Long Term Debt / Deuda a largo plazo
- p49100: Profit (Resultado del ejercicio)
- p40800: Amortization (Amortización) 


#### rraa_rrpp: Financial leveraging / apalancamiento financiero 
- p10000: Total Assets / Total activos
- p20000: Own Capital / Patrimonio neto

In [10]:
df = df[['cnae','p49100_h1','p40800_h1','p40100_mas_40500_h1','p31200_h1','p32300_h1', 'p10000_h1', 'p20000_h1',
         'target_status']]

In [11]:
df.head(5)

,cnae,p49100_h1,p40800_h1,p40100_mas_40500_h1,p31200_h1,p32300_h1,p10000_h1,p20000_h1,target_status
0,4110,-327182.94800,-86.48800,37471.84000,69756.01500,210339.37700,50490.12200,-250573.20000,1
1,2229,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1
2,0893,5535.46300,-705.34900,37718.34300,23.25000,24.71600,32122.56600,24320.40300,0
3,4931,3587.81983,-2781.84965,31213.27833,9075.93212,4550.94043,40073.08716,19569.74711,0
4,6202,667.30630,-111.25868,39523.88435,0.00000,0.00000,32436.04469,17485.63320,0


Dado que en el pipeline calcularemos algunos ratios (véase a continuación), debemos eliminar aquellas empresas que tengan valor cero en alguno de los denominadores (o también en el argumento del logaritmo en este caso). Creamos una columna con la suma de p49100_h1 y p40800_h1 para simplificar el trabajo.

        X['ebitda_income'] = (X.p49100_h1+X.p40800_h1)/(X.p40100_mas_40500_h1) 
        X['debt_ebitda'] =(X.p31200_h1 + X.p32300_h1) /(X.p49100_h1+X.p40800_h1) 
        X['rraa_rrpp'] = (X.p10000_h1 - X.p20000_h1) /X.p20000_h1
        X['log_operating_income'] = np.log(X.p40100_mas_40500_h1)

In [12]:
df.head()

,cnae,p49100_h1,p40800_h1,p40100_mas_40500_h1,p31200_h1,p32300_h1,p10000_h1,p20000_h1,target_status
0,4110,-327182.94800,-86.48800,37471.84000,69756.01500,210339.37700,50490.12200,-250573.20000,1
1,2229,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1
2,0893,5535.46300,-705.34900,37718.34300,23.25000,24.71600,32122.56600,24320.40300,0
3,4931,3587.81983,-2781.84965,31213.27833,9075.93212,4550.94043,40073.08716,19569.74711,0
4,6202,667.30630,-111.25868,39523.88435,0.00000,0.00000,32436.04469,17485.63320,0


In [13]:
df.loc[:,'p49100_mas_40800_h1'] = df['p49100_h1'] + df['p40800_h1']

/home/rafael/Programas/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:844: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/home/rafael/Programas/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [14]:
df.shape

(50000, 10)

In [15]:
df = df[(df['p40100_mas_40500_h1'] != 0) | (df['p49100_mas_40800_h1'] != 0) | (df['p20000_h1'] !=0 )]

In [16]:
df.shape

(32749, 10)

In [17]:
df.head()

,cnae,p49100_h1,p40800_h1,p40100_mas_40500_h1,p31200_h1,p32300_h1,p10000_h1,p20000_h1,target_status,p49100_mas_40800_h1
0,4110,-327182.94800,-86.48800,37471.84000,69756.01500,210339.37700,50490.12200,-250573.20000,1,-327269.43600
2,0893,5535.46300,-705.34900,37718.34300,23.25000,24.71600,32122.56600,24320.40300,0,4830.11400
3,4931,3587.81983,-2781.84965,31213.27833,9075.93212,4550.94043,40073.08716,19569.74711,0,805.97018
4,6202,667.30630,-111.25868,39523.88435,0.00000,0.00000,32436.04469,17485.63320,0,556.04762
5,4511,480.13643,-364.88315,35161.13851,1344.15659,5898.30420,15613.93227,6380.88943,0,115.25328


Eliminamos la variable adicional que creamos para limpiar el dataset y guardamos el resultado limpio en un archivo csv que se empleará posteriormente en el pipeline para entrenar el modelo con un conjunto de entrenamiento y comprobar su rendimiento sobre un conjunto de test.

In [18]:
df.drop(columns='p49100_mas_40800_h1', inplace=True)

Podemos buscar empresas que tuvieran valor 0 en las variables h1 (el año anterior) pero no en h2, indicando que habrían quebrado no este año, sino el precedente, para incluir tambiñen dichos datos en el modelo. Lo realizamos a continuación.

In [19]:
df_h2 = big_df

df_h2.loc[:,'p49100_mas_40800_h1'] = df_h2['p49100_h1'] + df_h2['p40800_h1']
df_h2.loc[:,'p49100_mas_40800_h2'] = df_h2['p49100_h2'] + df_h2['p40800_h2']

In [20]:
df_h2 = df_h2[((df_h2['p40100_mas_40500_h1'] == 0) & (df_h2['p40100_mas_40500_h2'] != 0))
               | ((df_h2['p40100_mas_40500_h1'] == 0) & (df_h2['p49100_mas_40800_h2'] != 0))
               | ((df_h2['p20000_h1'] == 0) & (df_h2['p20000_h2'] != 0))]

In [21]:
df_h2.head()

,id,nombre_x,nif,nombre,provincia,calle,telefono,web,desc_actividad,cnae,...,p40800_h1,p40800_h2,p49100_h0,p49100_h1,p49100_h2,h0_anio,nif_normalizado,target_status,p49100_mas_40800_h1,p49100_mas_40800_h2
26,27,Inversiones Y Explotaciones Turisticas SA,A28103182,Inversiones Y Explotaciones Turisticas SA,Madrid,"CALLE MAURICIO LEGENDRE, 16 - ENTREPLANTA",+34 97/1224400,www.melia.com,Gestión y explotación de hoteles.,5510,...,0.0,-2293.51565,0.00000,0.0,7992.25870,2017,28103182,0,0.0,5698.74305
53,54,Efron Consulting SL (Extinguida),B81626913,Efron Consulting SL (Extinguida),Madrid,"CALLE ULISES, 97",+34 91/4562937,www.efronconsulting.com,Consultoría informática y desarrollo de progra...,6202,...,0.0,-201.04841,0.00000,0.0,2248.49567,2017,81626913,1,0.0,2047.44726
75,76,Clinica Vistahermosa SA (Extinguida),A03009867,Clinica Vistahermosa SA (Extinguida),Alicante,"AVENIDA DENIA, 103",+34 96/5235100,www.clinicavistahermosa.es,PRESTACION DE SERVICIOS DE ASISTENCIA CLINICA ...,8610,...,0.0,-1375.95503,0.00000,0.0,1913.30567,2017,03009867,1,0.0,537.35064
102,103,Cooperativa Mesenor SCL,F40033276,Cooperativa Mesenor SCL,Segovia,"CARRETERA A-601, 601 - A 601 SALIDA 89",+34 92/1560999,,Fábrica de piensos.,0150,...,0.0,-334.57323,0.00000,0.0,572.33139,2017,40033276,0,0.0,237.75816
107,108,"Saica Flexible, Sociedad Anonima.",A50013952,"Saica Flexible, Sociedad Anonima.",Zaragoza,CALLE E (POLIGONO INDUSTRIAL MALPICA) (POL. IN...,+34 97/6465959,www.saica.com,Fabricación de envases y embalajes. Empresa de...,2222,...,0.0,-901.93668,1643.19898,0.0,3117.26964,2017,50013952,0,0.0,2215.33296


In [22]:
df_h2 = df_h2[['cnae','p49100_h2','p40800_h2','p40100_mas_40500_h2','p31200_h2','p32300_h2', 'p10000_h2', 'p20000_h2',
         'target_status']]

Renombramos las columnas como h1 aunque no se estrictamente cierto para concatenar estos dataframes con el anterior.

In [23]:
df_h2.columns = ['cnae','p49100_h1','p40800_h1','p40100_mas_40500_h1','p31200_h1','p32300_h1', 'p10000_h1', 'p20000_h1',
         'target_status']

In [24]:
df_h2.head(5)

,cnae,p49100_h1,p40800_h1,p40100_mas_40500_h1,p31200_h1,p32300_h1,p10000_h1,p20000_h1,target_status
26,5510,7992.25870,-2293.51565,37384.44669,48.93670,1849.58083,70706.37328,61126.65575,0
53,6202,2248.49567,-201.04841,37307.85620,788.24504,5321.11092,14679.50047,4965.08016,1
75,8610,1913.30567,-1375.95503,37233.94004,1297.26221,800.76152,29813.45813,22660.13716,1
102,0150,572.33139,-334.57323,37147.80542,480.88588,88.85362,9136.59467,3882.03827,0
107,2222,3117.26964,-901.93668,30719.46774,479.35028,229.37286,20264.94238,5554.29025,0


In [25]:
df_h2.shape

(1652, 9)

In [26]:
df.head(5)

,cnae,p49100_h1,p40800_h1,p40100_mas_40500_h1,p31200_h1,p32300_h1,p10000_h1,p20000_h1,target_status
0,4110,-327182.94800,-86.48800,37471.84000,69756.01500,210339.37700,50490.12200,-250573.20000,1
2,0893,5535.46300,-705.34900,37718.34300,23.25000,24.71600,32122.56600,24320.40300,0
3,4931,3587.81983,-2781.84965,31213.27833,9075.93212,4550.94043,40073.08716,19569.74711,0
4,6202,667.30630,-111.25868,39523.88435,0.00000,0.00000,32436.04469,17485.63320,0
5,4511,480.13643,-364.88315,35161.13851,1344.15659,5898.30420,15613.93227,6380.88943,0


In [27]:
df_global = pd.concat([df, df_h2], ignore_index=True)

In [28]:
df_global.shape

(34401, 9)

In [29]:
df_global.head()

,cnae,p49100_h1,p40800_h1,p40100_mas_40500_h1,p31200_h1,p32300_h1,p10000_h1,p20000_h1,target_status
0,4110,-327182.94800,-86.48800,37471.84000,69756.01500,210339.37700,50490.12200,-250573.20000,1
1,0893,5535.46300,-705.34900,37718.34300,23.25000,24.71600,32122.56600,24320.40300,0
2,4931,3587.81983,-2781.84965,31213.27833,9075.93212,4550.94043,40073.08716,19569.74711,0
3,6202,667.30630,-111.25868,39523.88435,0.00000,0.00000,32436.04469,17485.63320,0
4,4511,480.13643,-364.88315,35161.13851,1344.15659,5898.30420,15613.93227,6380.88943,0


In [30]:
df_global['target_status'].value_counts()

0    30795
1     3606
Name: target_status, dtype: int64

Vemos que la variable objetivo está bastante desbalanceada, ya que la mayoría de las empresas aún están activas, pero aún así, hemos conseguido aumentar la proporción de empresas qubreadas con respecto a los datos del último año exclusivamente.

In [31]:
df['target_status'].value_counts()

0    29922
1     2827
Name: target_status, dtype: int64

Contamos el número de elementos nulos en cada columna, si los hubiera.

In [32]:
df_global.isna().sum()

cnae                   0
p49100_h1              0
p40800_h1              0
p40100_mas_40500_h1    0
p31200_h1              0
p32300_h1              0
p10000_h1              0
p20000_h1              0
target_status          0
dtype: int64

In [33]:
df_global.describe()

,p49100_h1,p40800_h1,p40100_mas_40500_h1,p31200_h1,p32300_h1,p10000_h1,p20000_h1,target_status
count,3.440100e+04,3.440100e+04,3.440100e+04,3.440100e+04,3.440100e+04,3.440100e+04,3.440100e+04,34401.000000
mean,5.650094e+03,-3.261541e+03,8.253656e+04,2.132623e+04,9.340270e+03,3.674813e+05,7.248551e+04,0.104823
std,1.015451e+05,6.456527e+04,7.451809e+05,4.298018e+05,2.154461e+05,1.064197e+07,1.030922e+06,0.306329
min,-6.155635e+06,-9.649000e+06,-6.880100e+04,0.000000e+00,0.000000e+00,7.308400e-01,-6.779968e+06,0.000000
25%,1.259930e+02,-7.960008e+02,8.118816e+03,0.000000e+00,2.295335e+01,5.205103e+03,1.342936e+03,0.000000
50%,5.164490e+02,-1.976960e+02,1.399397e+04,2.660497e+02,5.421695e+02,1.182353e+04,4.190377e+03,0.000000
75%,1.872875e+03,-4.630775e+01,3.356200e+04,1.968699e+03,2.520769e+03,3.676177e+04,1.415400e+04,0.000000
max,5.469000e+06,0.000000e+00,5.257400e+07,4.561200e+07,2.253902e+07,1.339125e+09,1.026990e+08,1.000000


Vemos que todas las variables presentan resultados y límites dentro de lo aceptable. Sabemos que no existen nulos ni elementos con valor 0 en aquellas variables que impedirían el correcto uso del modelo. Guardamos por tanto el resultado limpio en un archivo csv que se empleará posteriormente en el pipeline para entrenar el modelo con un conjunto de entrenamiento y comprobar su rendimiento sobre un conjunto de test.

SMOTENC

In [34]:
from imblearn.over_sampling import SMOTENC
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter
from imblearn.pipeline import Pipeline

Número inicial de observaciones para cada clase

In [35]:
counter = Counter(df_global['target_status'])
print(counter)

Counter({0: 30795, 1: 3606})


Se amplia el número de observaciones para la clase '1'

In [317]:
over = SMOTENC(sampling_strategy=1,k_neighbors=10, categorical_features=[0], n_jobs=-1)
under = RandomUnderSampler(sampling_strategy=1)
steps = [('o', over), ('u', under)]
pipeline = Pipeline(steps=steps)

In [318]:
X = df_global.iloc[:,:-1]
y = df_global['target_status']

In [319]:
X_res, y_res = pipeline.fit_resample(X, y)

Número de observaciones para cada clase tras aplicar la clase SMOTENC.

In [320]:
counter = Counter(y_res)
print(counter)

Counter({0: 30795, 1: 30795})


Aspecto del nuevo dataset

In [321]:
df_prueba = pd.concat([X_res, y_res], axis=1)

In [322]:
df_prueba.head()

,cnae,p49100_h1,p40800_h1,p40100_mas_40500_h1,p31200_h1,p32300_h1,p10000_h1,p20000_h1,target_status
0,0893,5535.46300,-705.34900,37718.34300,23.25000,24.71600,32122.56600,24320.40300,0
1,4931,3587.81983,-2781.84965,31213.27833,9075.93212,4550.94043,40073.08716,19569.74711,0
2,6202,667.30630,-111.25868,39523.88435,0.00000,0.00000,32436.04469,17485.63320,0
3,4511,480.13643,-364.88315,35161.13851,1344.15659,5898.30420,15613.93227,6380.88943,0
4,4646,1496.03300,-144.30700,34000.01000,68.60000,12.97100,15023.07100,9584.98200,0


Se filtra el nuevo dataset, descartando aquellos valores de CNAE que son '' (string vacío)

In [323]:
df_prueba = df_prueba[df_prueba['cnae'] != '']

La códigos de la columna del CNAE se expresa en términos de las clases A, B, C, ... a la que pertenecen

In [324]:
df_prueba['cnae'] = df_prueba.astype(int)

In [325]:
def conditions(df_prueba):
    if df_prueba['cnae'] < 510:
        return 'A'
    elif df_prueba['cnae'] >= 510 and df_prueba['cnae'] < 1011:
        return 'B'
    elif df_prueba['cnae'] >= 1011 and df_prueba['cnae'] < 3512:
        return 'C'
    elif df_prueba['cnae'] >= 3512 and df_prueba['cnae'] < 3600:
        return 'D'
    elif df_prueba['cnae'] >= 3600 and df_prueba['cnae'] < 4110:
        return 'E'
    elif df_prueba['cnae'] >= 4110 and df_prueba['cnae'] < 4511:
        return 'F'
    elif df_prueba['cnae'] >= 4511 and df_prueba['cnae'] < 4910:
        return 'G'
    elif df_prueba['cnae'] >= 4910 and df_prueba['cnae'] < 5510:
        return 'H'
    elif df_prueba['cnae'] >= 5510 and df_prueba['cnae'] < 5811:
        return 'I'
    elif df_prueba['cnae'] >= 5811 and df_prueba['cnae'] < 6411:
        return 'J'
    elif df_prueba['cnae'] >= 6411 and df_prueba['cnae'] < 6810:
        return 'K'
    elif df_prueba['cnae'] >= 6810 and df_prueba['cnae'] < 6910:
        return 'L'
    elif df_prueba['cnae'] >= 6910 and df_prueba['cnae'] < 7711:
        return 'M'
    elif df_prueba['cnae'] >= 7711 and df_prueba['cnae'] < 8411:
        return 'N'
    elif df_prueba['cnae'] >= 8411 and df_prueba['cnae'] < 8510:
        return 'O'
    elif df_prueba['cnae'] >= 8510 and df_prueba['cnae'] < 8610:
        return 'P'
    elif df_prueba['cnae'] >= 8610 and df_prueba['cnae'] < 9001:
        return 'Q'
    elif df_prueba['cnae'] >= 9001 and df_prueba['cnae'] < 9411:
        return 'R'
    elif df_prueba['cnae'] >= 9411 and df_prueba['cnae'] < 9700:
        return 'S'
    elif df_prueba['cnae'] >= 9700 and df_prueba['cnae'] < 9900:
        return 'T'
    elif df_prueba['cnae'] >= 9900:
        return 'U'
    else:
        return 'Unknown'
    
df_prueba['cnae_reduced'] = df_prueba.apply(conditions, axis=1)

Se hace un get_dummies

In [326]:
df_prueba = pd.get_dummies(df_prueba, prefix='cnae', prefix_sep='_')

In [327]:
df_prueba

,cnae,p49100_h1,p40800_h1,p40100_mas_40500_h1,p31200_h1,p32300_h1,p10000_h1,p20000_h1,target_status,cnae_A,...,cnae_K,cnae_L,cnae_M,cnae_N,cnae_O,cnae_P,cnae_Q,cnae_R,cnae_S,cnae_U
0,893,5535.463000,-705.349000,37718.343000,23.250000,24.716000,32122.566000,24320.403000,0,0,...,0,0,0,0,0,0,0,0,0,0
1,4931,3587.819830,-2781.849650,31213.278330,9075.932120,4550.940430,40073.087160,19569.747110,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6202,667.306300,-111.258680,39523.884350,0.000000,0.000000,32436.044690,17485.633200,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4511,480.136430,-364.883150,35161.138510,1344.156590,5898.304200,15613.932270,6380.889430,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4646,1496.033000,-144.307000,34000.010000,68.600000,12.971000,15023.071000,9584.982000,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61585,8010,190.635253,-390.737575,32798.039013,328.474351,1965.647523,12368.903754,1997.384875,1,0,...,0,0,0,1,0,0,0,0,0,0
61586,5629,3496.622354,-1956.471671,66439.847935,2941.372643,2081.902742,51220.017549,15459.364802,1,0,...,0,0,0,0,0,0,0,0,0,0
61587,5229,588.350455,-39.793738,16090.776945,0.000000,326.372279,5521.354871,3105.052098,1,0,...,0,0,0,0,0,0,0,0,0,0
61588,1013,1897.095650,-2018.670730,112153.794930,895.289200,9358.245070,20554.917570,2982.518750,1,0,...,0,0,0,0,0,0,0,0,0,0


In [328]:
df_final = df_prueba.drop(['cnae'], axis=1)

Se crean los conjustos de entrenamiento y de test.

In [329]:
from sklearn.model_selection import train_test_split
X = df_final.drop(['target_status'], axis=1)
y = df_final['target_status']
X_train, X_test, y_train, y_test = train_test_split( X, y , test_size = 0.2 , random_state = 42, stratify = y)

Clasificador Random Forest

In [330]:
from sklearn.ensemble import RandomForestClassifier
rf_class = RandomForestClassifier(n_estimators=1000 , 
                       max_depth=20, 
                       random_state = 1234,
                       class_weight={0:0.1, 1:0.9}, n_jobs=-1)

In [331]:
model = rf_class.fit(X_train, y_train)


y_pred = model.predict( X_train ) 
y_pred_test = model.predict( X_test ) 

from sklearn.metrics import accuracy_score
print("Accuracy train: {0}".format(accuracy_score(y_pred,y_train)))

print("Accuracy test: {0}".format(accuracy_score(y_pred_test,y_test)))

Accuracy train: 0.6409449777759737
Accuracy test: 0.5966065919792174


In [332]:
from sklearn.metrics import classification_report

print("classification report for train")
print(classification_report(y_train, y_pred))

print("classification report for test")
print(classification_report(y_test, y_pred_test))

classification report for train
              precision    recall  f1-score   support

           0       1.00      0.28      0.44     24635
           1       0.58      1.00      0.74     24636

    accuracy                           0.64     49271
   macro avg       0.79      0.64      0.59     49271
weighted avg       0.79      0.64      0.59     49271

classification report for test
              precision    recall  f1-score   support

           0       0.91      0.21      0.35      6159
           1       0.55      0.98      0.71      6159

    accuracy                           0.60     12318
   macro avg       0.73      0.60      0.53     12318
weighted avg       0.73      0.60      0.53     12318



In [333]:
import numpy as np
from sklearn import metrics
fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred_test)
metrics.auc(fpr, tpr)

0.5966065919792174